In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation
from sklearn.metrics import accuracy_score

# Step 1: Load the dataset
df = pd.read_csv("WineQT.csv")

# Step 2: Drop the 'Id' column (not needed)
df.drop(columns=["Id"], inplace=True)

# Step 3: Separate features (X) and target (y)
X = df.drop(columns=["quality"])  # Features
y = df["quality"]  # Target variable (discrete classes)

# Step 4: Map target labels to a range of [0, num_classes - 1]
unique_classes = y.unique()
num_classes = len(unique_classes)
label_mapping = {label: idx for idx, label in enumerate(sorted(unique_classes))}
y_mapped = y.map(label_mapping)

# Step 5: Perform Z-score normalization (standardization)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert the scaled features back to a DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Step 6: Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y_mapped, test_size=0.2, random_state=42)

# Step 7: Define the base model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    BatchNormalization(), 
    Dense(16, activation='relu'),  # Hidden layer
    BatchNormalization(), 
    Dense(8, activation='relu'),  # Hidden layer
    Dense(num_classes, activation='softmax')  # Output layer (num_classes)
])

# Step 8: Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 9: Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

# Step 10: Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = tf.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred)

print(f"Base Model Accuracy: {accuracy:.4f}")

Epoch 1/50
12/12 [==============================] - 2s 41ms/step - loss: 2.1738 - accuracy: 0.0821 - val_loss: 1.8061 - val_accuracy: 0.2240
Epoch 2/50
12/12 [==============================] - 0s 9ms/step - loss: 1.9487 - accuracy: 0.1382 - val_loss: 1.7431 - val_accuracy: 0.2896
Epoch 3/50
12/12 [==============================] - 0s 12ms/step - loss: 1.8080 - accuracy: 0.1833 - val_loss: 1.7003 - val_accuracy: 0.3661
Epoch 4/50
12/12 [==============================] - 0s 11ms/step - loss: 1.7023 - accuracy: 0.2408 - val_loss: 1.6683 - val_accuracy: 0.4098
Epoch 5/50
12/12 [==============================] - 0s 10ms/step - loss: 1.6469 - accuracy: 0.2982 - val_loss: 1.6409 - val_accuracy: 0.4372
Epoch 6/50
12/12 [==============================] - 0s 11ms/step - loss: 1.5789 - accuracy: 0.3283 - val_loss: 1.6148 - val_accuracy: 0.4426
Epoch 7/50
12/12 [==============================] - 0s 11ms/step - loss: 1.5414 - accuracy: 0.3529 - val_loss: 1.5853 - val_accuracy: 0.4863
Epoch 8/50
12/

### **Changes Made to the Base Model**
1. **Added Batch Normalization**:
   - Added `BatchNormalization` after each dense layer (except the output layer).
   - Batch Normalization stabilizes training by normalizing the outputs of each layer.

2. **Reduced Layer Sizes**:
   - Reduced the number of neurons in each layer by half:
     - Input layer: 64 → 32 neurons.
     - Hidden layer 1: 32 → 16 neurons.
     - Hidden layer 2: 16 → 8 neurons.

3. **Increased Batch Size**:
   - Increased the batch size from 32 to 64.

---

### **Impact of Changes**

#### **1. Batch Normalization**
- **Effect on Training**:
  - Batch Normalization helps stabilize training by reducing internal covariate shift.
  - It allows the model to use higher learning rates and converge faster.
- **Effect on Accuracy**:
  - In your case, the accuracy improved slightly from **60.26%** to **64.63%**.
  - This suggests that Batch Normalization helped the model generalize better.

#### **2. Reduced Layer Sizes**
- **Effect on Training**:
  - Smaller layers reduce the model’s capacity, which can help prevent overfitting, especially on smaller datasets like the Wine Quality dataset.
  - However, if the layers are too small, the model may underfit and fail to capture complex patterns in the data.
- **Effect on Accuracy**:
  - The accuracy improved slightly, indicating that the smaller layers were sufficient for this dataset.

#### **3. Increased Batch Size**
- **Effect on Training**:
  - A larger batch size (64 vs. 32) provides a more accurate estimate of the gradient, which can lead to smoother convergence.
  - However, larger batch sizes may require more epochs to converge, as each update is less frequent.
- **Effect on Accuracy**:
  - The accuracy improved slightly, suggesting that the larger batch size helped the model generalize better.

---

### **Comparison of Results**

| **Metric**                | **Base Model**               | **Improved Model**            |
|---------------------------|------------------------------|--------------------------------|
| **Input Layer**            | 64 neurons                  | 32 neurons                    |
| **Hidden Layer 1**         | 32 neurons                  | 16 neurons                    |
| **Hidden Layer 2**         | 16 neurons                  | 8 neurons                     |
| **Batch Normalization**    | No                          | Yes                           |
| **Batch Size**             | 32                          | 64                            |
| **Training Accuracy**      | ~77.84%                     | ~69.36%                       |
| **Validation Accuracy**    | ~54.64%                     | ~55.74%                       |
| **Test Accuracy**          | 60.26%                      | 64.63%                        |

---

### **Key Observations**
1. **Improved Generalization**:
   - The improved model achieved higher test accuracy (**64.63%**) compared to the base model (**60.26%**).
   - This suggests that the changes (Batch Normalization, smaller layers, larger batch size) helped the model generalize better to unseen data.

2. **Training Stability**:
   - Batch Normalization made the training process more stable, as seen in the smoother loss curves.

3. **Reduced Overfitting**:
   - The smaller layers and larger batch size likely reduced overfitting, as the validation accuracy remained closer to the training accuracy.

4. **Convergence Speed**:
   - The improved model converged slightly faster, as Batch Normalization allows for higher learning rates and smoother gradient updates.

---

### **Conclusion**
The changes you made to the base model (adding Batch Normalization, reducing layer sizes, and increasing the batch size) had a **positive impact** on the model’s performance. Specifically:
- **Accuracy**: Improved from **60.26%** to **64.63%**.
- **Generalization**: The model performed better on the test set, indicating improved generalization.
- **Training Stability**: Batch Normalization stabilized training and allowed for smoother convergence.

These changes align with best practices for training neural networks on smaller datasets, where reducing overfitting and stabilizing training are critical.
